In [1]:
import pandas as pd
import numpy as np
import joblib
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

import pickle

In [2]:
# モデルの読み込み（ファイル名は例）
lgb_model = joblib.load('./output/モデル/lgb_model.pkl')
nn_model = joblib.load('./output/モデル/nn_model.pkl')
cat_model = joblib.load('./output/モデル/cat_model.pkl')
meta_model = joblib.load('./output/モデル/meta_model.pkl')

In [3]:
train_df = pd.read_csv('./output/中間データ/test_preprocessed.csv')
train_df.head()

,datetime,y,week,soldout,name,kcal,remarks,payday,weather,precipitation,...,y_rolling_mean_7,y_rolling_max_7,y_rolling_min_7,y_rolling_mean_14,y_rolling_max_14,y_rolling_min_14,kcal_missing_flag,weather_group,event_キャリアアップ支援セミナー,event_ママの会
0,2014-10-01,NaN,-0.166787,1,メンチカツ,-0.079503,0,0.0,1.376831,-0.313559,...,46.285714,59.0,29.0,54.285714,115.0,29.0,0,1.626160,False,False
1,2014-10-02,NaN,0.574488,0,バーベキューチキン,-0.446438,0,0.0,0.736444,-0.340302,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0.350740,False,False
2,2014-10-03,NaN,1.315764,0,豚肉のマスタード焼き,-1.180308,0,0.0,-0.544328,-0.340302,...,NaN,NaN,NaN,NaN,NaN,NaN,0,-0.924679,False,False
3,2014-10-06,NaN,-1.649338,1,麻婆春雨,-1.547242,0,0.0,1.376831,1.023580,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1.626160,False,False
4,2014-10-07,NaN,-0.908062,0,厚揚げ肉みそ炒め,0.654367,0,0.0,-0.544328,-0.340302,...,NaN,NaN,NaN,NaN,NaN,NaN,0,-0.924679,False,False


In [4]:
X_test = train_df.drop(columns=['name', 'datetime'], errors='ignore').astype(np.float32)

In [5]:
with open('./output/モデル/cat_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

y_pred = loaded_model.predict(X_test)

In [7]:
# 3. 特徴量の順序を学習時と合わせる（lgb_model を使って）
expected_cols = lgb_model.feature_name_

# 欠損している列を補完（0などで埋める）
for col in expected_cols:
    if col not in X_test.columns:
        X_test[col] = 0

X_test = X_test[expected_cols]

# 4. 各モデルで予測
lgb_pred = lgb_model.predict(X_test)
cat_pred = cat_model.predict(X_test)
nn_pred = nn_model.predict(X_test).flatten()

# NaN を補完（推奨: 0で埋める）
lgb_pred = np.nan_to_num(lgb_pred, nan=0.0)
cat_pred = np.nan_to_num(cat_pred, nan=0.0)
nn_pred = np.nan_to_num(nn_pred, nan=0.0)

# 5. スタッキング用特徴量を作成
X_meta_test = np.vstack([lgb_pred, cat_pred, nn_pred]).T

# 6. メタモデルで最終予測
final_pred = meta_model.predict(X_meta_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


#### 予測結果の出力

In [8]:
result_df = pd.read_csv('./input/sample.csv', header=None)

In [9]:
result_df.iloc[:, 1] = final_pred

/var/folders/sb/t91v_nzj05sbsfnhcjb832xr0000gn/T/ipykernel_3932/1831291914.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[101.99834432  82.26051505  75.82080122  62.45467128  62.14955766
  63.63233266  62.61746211  98.26848235  56.77423524  91.51343173
  73.24234956  64.89861661  64.30441803  65.17400716  90.90647235
  85.07972214  99.21148989  62.6207982   67.59798117  66.99993353
  66.4334357   72.98906973  71.30953423  91.45070902  67.46522705
 100.71855061  65.04820865  90.58471977  89.88735685  67.95536102
  84.43223178  97.82640105  93.36963545  91.41955934  83.25824488
 109.11305158  86.71778086 100.16998944  83.64141109  85.91109867]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result_df.iloc[:, 1] = final_pred


In [10]:
result_df.to_csv('output/予測結果/予測結果.csv', index=False)